# External validation: ROC & PR Curve comparison with confidence intervals

Confidence are obtained from variance of performance across folds

In [ ]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import pandas as pd
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerTuple

In [ ]:
# for some reason spacing of plots only works correctly in opsum_shap environment 
# verify that python version is 3.7
import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 7

In [ ]:

transformer_ext_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_Death/external_validation'
lstm_ext_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/external_validation/3M_Death/2023_01_04_2020/all_folds'
xgb_ext_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_Death/external_validation/all_folds'
thrive_c_ext_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_3m_mimic_death_predictions/3m_mimic_death_test_gt_and_pred.pkl'

transformer_ext_death_in_hosp_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/Death_in_hospital/external_validation'
lstm_ext_death_in_hosp_predictions_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/external_validation/Death_in_hospital/2024_02_24'
xgb_ext_death_in_hosp_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/Death_in_hospital/external_validation'
thrive_c_ext_death_in_hosp_predictions_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_death_in_hospital_mimic_predictions/mimic_Death_in_hospital_test_gt_and_pred.pkl'

roc_auc_statistics_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/stroke_research/geneva_stroke_unit_dataset/opsum_paper/supp_figures_and_tables/model_comparison/table/revision/overall_model_comparison.csv'

In [ ]:
output_dir = '/Users/jk1/Downloads'

In [ ]:
n_folds = 5
seed = 42

# for transformer, indices already start at 0
death_transformer_selected_fold = 1
death_in_hosp_transformer_selected_fold = 2

# subtract one to go from fold number to index (as folds will be in a list)
death_xgb_selected_fold = 3 - 1
death_in_hosp_xgb_selected_fold = 3 - 1

# subtract one to go from fold number to index (as folds will be in a list)
death_lstm_selected_fold = 2 - 1
death_in_hosp_lstm_selected_fold = 3 - 1

Load data

In [ ]:
roc_auc_statistics = pd.read_csv(roc_auc_statistics_path)
roc_auc_statistics['clean_roc_auc'] = roc_auc_statistics['ROC AUC'].apply(lambda x: float(x.split(' ')[0]))

In [ ]:
thrivec_ext_death_gt, thrivec_ext_death_predictions = pickle.load(open(thrive_c_ext_death_predictions_path, 'rb'))
throvec_ext_death_in_hosp_gt, thrivec_ext_death_in_hosp_predictions = pickle.load(open(thrive_c_ext_death_in_hosp_predictions_path, 'rb'))

thrivec_ext_death_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'THRIVE-C') & (roc_auc_statistics.Outcome == '3M death')
                                                     & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]
thrivec_ext_death_in_hosp_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'THRIVE-C') & (roc_auc_statistics.Outcome == 'Death in hospital')
                                                        & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]

In [ ]:
transformer_ext_death_folds = []
for fidx in range(n_folds):
    transformer_ext_death_folds.append(pickle.load(open(os.path.join(transformer_ext_death_predictions_path, f'fold_{fidx}_test_gt_and_pred.pkl'), 'rb')))
    
transformer_ext_death_in_hosp_folds = []
for fidx in range(n_folds):
    transformer_ext_death_in_hosp_folds.append(pickle.load(open(os.path.join(transformer_ext_death_in_hosp_predictions_path, f'fold_{fidx}_test_gt_and_pred.pkl'), 'rb')))

transformer_ext_death_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'Transformer') & (roc_auc_statistics.Outcome == '3M Death')
                                                     & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]    
transformer_ext_death_in_hosp_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'Transformer') & (roc_auc_statistics.Outcome == 'Death in hospital')
                                                        & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]

In [ ]:
xgb_ext_death_folds = []
for fidx in range(n_folds):
    xgb_ext_death_folds.append(pickle.load(open(os.path.join(xgb_ext_death_predictions_path, f'external_validation_gt_and_pred_cv_{fidx}.pkl'), 'rb')))
    
xgb_ext_death_in_hosp_folds = []
for fidx in range(n_folds):
    xgb_ext_death_in_hosp_folds.append(pickle.load(open(os.path.join(xgb_ext_death_in_hosp_predictions_path, f'external_validation_gt_and_pred_cv_{fidx}.pkl'), 'rb')))

xgb_ext_death_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'XGB') & (roc_auc_statistics.Outcome == '3M Death')
                                                     & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]

xgb_ext_death_in_hosp_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'XGB') & (roc_auc_statistics.Outcome == 'Death in hospital')
                                                        & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]

In [ ]:
lstm_ext_death_folds = []
for fidx in range(n_folds):
    # search in every subdir of lstm_mrs02_predictions_path to find and load file: test_gt_and_pred_fold_{fidx}.pkl
    for subdir in os.listdir(lstm_ext_death_predictions_path):
        if os.path.isdir(os.path.join(lstm_ext_death_predictions_path, subdir)):
            for file in os.listdir(os.path.join(lstm_ext_death_predictions_path, subdir)):
                if file == f'test_gt_and_pred_fold_{fidx+1}.pkl':
                    lstm_ext_death_folds.append(pickle.load(open(os.path.join(lstm_ext_death_predictions_path, subdir, file), 'rb')))
                    
lstm_ext_death_in_hosp_folds = []
for fidx in range(n_folds):
    # search in every subdir of lstm_mrs02_predictions_path to find and load file: test_gt_and_pred_fold_{fidx}.pkl
    for subdir in os.listdir(lstm_ext_death_in_hosp_predictions_path):
        if os.path.isdir(os.path.join(lstm_ext_death_in_hosp_predictions_path, subdir)):
            for file in os.listdir(os.path.join(lstm_ext_death_in_hosp_predictions_path, subdir)):
                if file == f'test_gt_and_pred_fold_{fidx+1}.pkl':
                    lstm_ext_death_in_hosp_folds.append(pickle.load(open(os.path.join(lstm_ext_death_in_hosp_predictions_path, subdir, file), 'rb')))

lstm_ext_death_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'LSTM') & (roc_auc_statistics.Outcome == '3M Death')
                                                     & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]

lstm_ext_death_in_hosp_roc_auc = roc_auc_statistics[(roc_auc_statistics.Model == 'LSTM') & (roc_auc_statistics.Outcome == 'Death in hospital')
                                                        & (roc_auc_statistics.Dataset == 'MIMIC')]['clean_roc_auc'].values[0]

In [ ]:
all_colors_palette = sns.color_palette(['#f61067', '#049b9a', '#012D98', '#a76dfe'], n_colors=4)
all_colors_palette

# Helper functions

In [ ]:
def compute_roc_and_pr_curves(folds, n_interpolated_points=200):
    roc_df = pd.DataFrame()
    resampled_roc_df = pd.DataFrame()
    roc_aucs = []
    pr_df = pd.DataFrame()
    resampled_pr_df = pd.DataFrame()
    pr_aucs = []
    for fidx in tqdm(range(n_folds)):
        fpr, tpr, _ = roc_curve(folds[fidx][0], folds[fidx][1])
        roc_aucs.append(auc(fpr, tpr))
        resampled_tpr = np.interp(np.linspace(0, 1, n_interpolated_points), fpr, tpr)
        roc_df = roc_df.append(pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'fold': fidx}))
        resampled_roc_df = resampled_roc_df.append(pd.DataFrame({'fpr': np.linspace(0,1,n_interpolated_points),  'tpr': resampled_tpr, 'fold': fidx}))

        precision, recall, _ = precision_recall_curve(folds[fidx][0], folds[fidx][1])
        recall, precision = zip(*sorted(zip(recall, precision)))
        pr_aucs.append(auc(recall, precision))
        resampled_precision = np.interp(np.linspace(0, 1, n_interpolated_points), recall, precision)
        pr_df = pr_df.append(pd.DataFrame({'recall': recall, 'precision': precision, 'fold': fidx}))
        resampled_pr_df = resampled_pr_df.append(pd.DataFrame({'recall': np.linspace(0,1,n_interpolated_points),  'precision': resampled_precision, 'fold': fidx}))

    return roc_df, resampled_roc_df, roc_aucs, pr_df, resampled_pr_df, pr_aucs


## Prepare data for death (3 months) external validation

In [ ]:
# split thrivec data into 5 folds
ext_death_thrivec_folds = []
for fidx in range(n_folds):
    ext_death_thrivec_folds.append((thrivec_ext_death_gt[fidx::n_folds], thrivec_ext_death_predictions[fidx::n_folds]))

ext_death_thrivec_fpr, ext_death_thrivec_tpr, _ = roc_curve(thrivec_ext_death_gt, thrivec_ext_death_predictions)
ext_death_thrivec_roc_auc = auc(ext_death_thrivec_fpr, ext_death_thrivec_tpr)
ext_death_thrivec_resampled_tpr = np.interp(np.linspace(0, 1, 200), ext_death_thrivec_fpr, ext_death_thrivec_tpr)

ext_death_thrivec_precision, ext_death_thrivec_recall, _ = precision_recall_curve(thrivec_ext_death_gt, thrivec_ext_death_predictions)
ext_death_thrivec_recall, ext_death_thrivec_precision = zip(*sorted(zip(ext_death_thrivec_recall, ext_death_thrivec_precision)))
ext_death_thrivec_pr_auc = auc(ext_death_thrivec_recall, ext_death_thrivec_precision)
ext_death_thrivec_resampled_precision = np.interp(np.linspace(0, 1, 200), ext_death_thrivec_recall, ext_death_thrivec_precision)

ext_death_thrivec_roc_df, ext_death_thrivec_resampled_roc_df, ext_death_thrivec_roc_aucs, ext_death_thrivec_pr_df, ext_death_thrivec_resampled_pr_df, ext_death_thrivec_pr_aucs = compute_roc_and_pr_curves(
    ext_death_thrivec_folds)

ext_death_thrivec_resampled_roc_std = ext_death_thrivec_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
ext_death_thrivec_resampled_pr_std = ext_death_thrivec_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision'][
    'std']

In [ ]:
transformer_ext_death_roc_df, transformer_ext_death_resampled_roc_df, transformer_ext_death_roc_aucs, transformer_ext_death_pr_df, transformer_ext_death_resampled_pr_df, transformer_ext_death_pr_aucs = compute_roc_and_pr_curves(transformer_ext_death_folds)

transformer_ext_death_resampled_roc_std = transformer_ext_death_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
transformer_ext_death_resampled_pr_std = transformer_ext_death_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision']['std']

In [ ]:
death_ext_xgb_roc_df, death_ext_xgb_resampled_roc_df, death_ext_xgb_roc_aucs, death_ext_xgb_pr_df, death_ext_xgb_resampled_pr_df, death_ext_xgb_pr_aucs = compute_roc_and_pr_curves(xgb_ext_death_folds)
death_ext_xgb_resampled_roc_std = death_ext_xgb_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
death_ext_xgb_resampled_pr_std = death_ext_xgb_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision']['std']

In [ ]:
ext_death_lstm_roc_df, ext_death_lstm_resampled_roc_df, ext_death_lstm_roc_aucs, ext_death_lstm_pr_df, ext_death_lstm_resampled_pr_df, ext_death_lstm_pr_aucs = compute_roc_and_pr_curves(lstm_ext_death_folds)
ext_death_lstm_resampled_roc_std = ext_death_lstm_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
ext_death_lstm_resampled_pr_std = ext_death_lstm_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision']['std']

## Prepare data for death in hospital external validation

In [ ]:
# split thrivec data into 5 folds
ext_death_in_hosp_thrivec_folds = []
for fidx in range(n_folds):
    ext_death_in_hosp_thrivec_folds.append((throvec_ext_death_in_hosp_gt[fidx::n_folds], thrivec_ext_death_in_hosp_predictions[fidx::n_folds]))
    
ext_death_in_hosp_thrivec_fpr, ext_death_in_hosp_thrivec_tpr, _ = roc_curve(throvec_ext_death_in_hosp_gt, thrivec_ext_death_in_hosp_predictions)
ext_death_in_hosp_thrivec_roc_auc = auc(ext_death_in_hosp_thrivec_fpr, ext_death_in_hosp_thrivec_tpr)
ext_death_in_hosp_thrivec_resampled_tpr = np.interp(np.linspace(0, 1, 200), ext_death_in_hosp_thrivec_fpr, ext_death_in_hosp_thrivec_tpr)

ext_death_in_hosp_thrivec_precision, ext_death_in_hosp_thrivec_recall, _ = precision_recall_curve(throvec_ext_death_in_hosp_gt, thrivec_ext_death_in_hosp_predictions)
ext_death_in_hosp_thrivec_recall, ext_death_in_hosp_thrivec_precision = zip(*sorted(zip(ext_death_in_hosp_thrivec_recall, ext_death_in_hosp_thrivec_precision)))

ext_death_in_hosp_thrivec_pr_auc = auc(ext_death_in_hosp_thrivec_recall, ext_death_in_hosp_thrivec_precision)
ext_death_in_hosp_thrivec_resampled_precision = np.interp(np.linspace(0, 1, 200), ext_death_in_hosp_thrivec_recall, ext_death_in_hosp_thrivec_precision)

ext_death_in_hosp_thrivec_roc_df, ext_death_in_hosp_thrivec_resampled_roc_df, ext_death_in_hosp_thrivec_roc_aucs, ext_death_in_hosp_thrivec_pr_df, ext_death_in_hosp_thrivec_resampled_pr_df, ext_death_in_hosp_thrivec_pr_aucs = compute_roc_and_pr_curves(
    ext_death_in_hosp_thrivec_folds)

ext_death_in_hosp_thrivec_resampled_roc_std = ext_death_in_hosp_thrivec_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
ext_death_in_hosp_thrivec_resampled_pr_std = ext_death_in_hosp_thrivec_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision'][
    'std']

In [ ]:
transformer_ext_death_in_hosp_roc_df, transformer_ext_death_in_hosp_resampled_roc_df, transformer_ext_death_in_hosp_roc_aucs, transformer_ext_death_in_hosp_pr_df, transformer_ext_death_in_hosp_resampled_pr_df, transformer_ext_death_in_hosp_pr_aucs = compute_roc_and_pr_curves(transformer_ext_death_in_hosp_folds)

transformer_ext_death_in_hosp_resampled_roc_std = transformer_ext_death_in_hosp_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
transformer_ext_death_in_hosp_resampled_pr_std = transformer_ext_death_in_hosp_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision']['std']

In [ ]:
death_ext_xgb_in_hosp_roc_df, death_ext_xgb_in_hosp_resampled_roc_df, death_ext_xgb_in_hosp_roc_aucs, death_ext_xgb_in_hosp_pr_df, death_ext_xgb_in_hosp_resampled_pr_df, death_ext_xgb_in_hosp_pr_aucs = compute_roc_and_pr_curves(xgb_ext_death_in_hosp_folds)
death_ext_xgb_in_hosp_resampled_roc_std = death_ext_xgb_in_hosp_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
death_ext_xgb_in_hosp_resampled_pr_std = death_ext_xgb_in_hosp_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision']['std']

In [ ]:
ext_death_lstm_in_hosp_roc_df, ext_death_lstm_in_hosp_resampled_roc_df, ext_death_lstm_in_hosp_roc_aucs, ext_death_lstm_in_hosp_pr_df, ext_death_lstm_in_hosp_resampled_pr_df, ext_death_lstm_in_hosp_pr_aucs = compute_roc_and_pr_curves(lstm_ext_death_in_hosp_folds)
ext_death_lstm_in_hosp_resampled_roc_std = ext_death_lstm_in_hosp_resampled_roc_df.groupby('fpr').agg({'tpr': ['mean', 'std']})['tpr']['std']
ext_death_lstm_in_hosp_resampled_pr_std = ext_death_lstm_in_hosp_resampled_pr_df.groupby('recall').agg({'precision': ['mean', 'std']})['precision']['std']

### Resampled ROC curve with fold variation


In [ ]:
def plot_roc_auc_curve(transformer_roc_df, transformer_resampled_roc_df, transformer_resampled_roc_std, transformer_roc_auc, transformer_selected_fold,
                            xgb_roc_df, xgb_resampled_roc_df, xgb_resampled_roc_std, xgb_roc_auc, xgb_selected_fold,
                            lstm_roc_df, lstm_resampled_roc_df, lstm_resampled_roc_std, lstm_roc_auc, lstm_selected_fold,
                           thrivec_resampled_tpr, thrivec_resampled_roc_std, thrivec_roc_auc,
                           ax, plot_legend = True, tick_label_size = 11, label_font_size = 13):

    all_colors_palette = sns.color_palette(['#f61067', '#049b9a', '#012D98', '#a76dfe'], n_colors=4)
    n_float_numbers = 3
    
    ## Main model: Transformer
    # plot variation across folds (+/- 1 std)
    error = 1*transformer_resampled_roc_std.values
    lower = transformer_resampled_roc_df[transformer_resampled_roc_df.fold == transformer_selected_fold].tpr - error
    upper = transformer_resampled_roc_df[transformer_resampled_roc_df.fold == transformer_selected_fold].tpr + error
    ax.fill_between(transformer_resampled_roc_std.index, lower, upper, alpha=0.2, color=all_colors_palette[0])
    
    # Plot selected fold in bold
    ax = sns.lineplot(data=transformer_roc_df[transformer_roc_df.fold == transformer_selected_fold], x='fpr', y='tpr', color=all_colors_palette[0], label=f'Transformer (area = %0.{n_float_numbers}f)' % transformer_roc_auc,
                       ax=ax, errorbar=None)
    
    
    ## Comparators: THRIVE-C
    # plot variation across folds (+/- 1 std)
    error = 1*thrivec_resampled_roc_std.values
    lower = thrivec_resampled_tpr - error
    upper = thrivec_resampled_tpr + error
    ax.fill_between(np.linspace(0, 1, 200), lower, upper, alpha=0.1, color=all_colors_palette[1])
    
    # Plot THRIVE-C in bold
    sns.lineplot(x=np.linspace(0, 1, 200), y=thrivec_resampled_tpr, color=all_colors_palette[1], label=f'THRIVE-C (area = %0.{n_float_numbers}f)' % thrivec_roc_auc,
                 ax=ax, linewidth=2)
    
    ## Comparators: XGBoost 
    error = 1*xgb_resampled_roc_std.values
    lower = xgb_resampled_roc_df[xgb_resampled_roc_df.fold == xgb_selected_fold].tpr - error
    upper = xgb_resampled_roc_df[xgb_resampled_roc_df.fold == xgb_selected_fold].tpr + error
    ax.fill_between(xgb_resampled_roc_std.index, lower, upper, alpha=0.2, color=all_colors_palette[2])   
    ax = sns.lineplot(data=xgb_roc_df[xgb_roc_df.fold == xgb_selected_fold], x='fpr', y='tpr', color=all_colors_palette[2], label=f'XGBoost (area = %0.{n_float_numbers}f)' % xgb_roc_auc,
                       ax=ax, errorbar=None)
    
    ## Comparators: LSTM
    error = 1*lstm_resampled_roc_std.values
    lower = lstm_resampled_roc_df[lstm_resampled_roc_df.fold == lstm_selected_fold].tpr - error
    upper = lstm_resampled_roc_df[lstm_resampled_roc_df.fold == lstm_selected_fold].tpr + error
    ax.fill_between(lstm_resampled_roc_std.index, lower, upper, alpha=0.2, color=all_colors_palette[3])
    ax = sns.lineplot(data=lstm_roc_df[lstm_roc_df.fold == lstm_selected_fold], x='fpr', y='tpr', color=all_colors_palette[3], label=f'LSTM (area = %0.{n_float_numbers}f)' % lstm_roc_auc,
                       ax=ax, errorbar=None)
    
    ax.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--', alpha=0.5)
    
    ax.set_xlabel('1 - Specificity (False Positive Rate)', fontsize=label_font_size)
    ax.set_ylabel('Sensitivity (True Positive Rate)', fontsize=label_font_size)
    ax.tick_params('x', labelsize=tick_label_size)
    ax.tick_params('y', labelsize=tick_label_size)
    
    if plot_legend:
        legend_markers, legend_labels = ax.get_legend_handles_labels()
        sd1_patch = mpatches.Patch(color=all_colors_palette[0], alpha=0.3)
        sd2_patch = mpatches.Patch(color=all_colors_palette[1], alpha=0.3)
        sd_3_patch = mpatches.Patch(color=all_colors_palette[2], alpha=0.3)
        sd_4_patch = mpatches.Patch(color=all_colors_palette[3], alpha=0.3)
        sd_marker = (sd1_patch, sd2_patch, sd_3_patch, sd_4_patch)
        sd_labels = '± s.d.'
        legend_markers.append(sd_marker)
        legend_labels.append(sd_labels)
        ax.legend(legend_markers, legend_labels, fontsize=label_font_size,
                  handler_map={tuple: HandlerTuple(ndivide=None)})
    
    else:
        # remove legend
        ax.get_legend().remove()
    
    fig = ax.get_figure()
    return fig

#### Plot external validation death (3 months) ROC curve

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False, 'figure.figsize':(10,10)}
sns.set_theme(style="whitegrid", rc=custom_params, context="paper", font_scale = 1)

ax = plt.subplot(111)

plot_roc_auc_curve(transformer_ext_death_roc_df, transformer_ext_death_resampled_roc_df, transformer_ext_death_resampled_roc_std, transformer_ext_death_roc_auc, death_transformer_selected_fold,
                          death_ext_xgb_roc_df, death_ext_xgb_resampled_roc_df, death_ext_xgb_resampled_roc_std, xgb_ext_death_roc_auc, death_xgb_selected_fold,
                            ext_death_lstm_roc_df, ext_death_lstm_resampled_roc_df, ext_death_lstm_resampled_roc_std, lstm_ext_death_roc_auc, death_lstm_selected_fold,
                       ext_death_thrivec_resampled_tpr, ext_death_thrivec_resampled_roc_std, thrivec_ext_death_roc_auc, 
                       ax, plot_legend = True, tick_label_size = 11, label_font_size = 13)

plt.show()

#### Plot external validation death in hospital ROC curve

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False, 'figure.figsize':(10,10)}
sns.set_theme(style="whitegrid", rc=custom_params, context="paper", font_scale = 1)

ax = plt.subplot(111)

plot_roc_auc_curve(transformer_ext_death_in_hosp_roc_df, transformer_ext_death_in_hosp_resampled_roc_df, transformer_ext_death_in_hosp_resampled_roc_std, transformer_ext_death_in_hosp_roc_auc, death_in_hosp_transformer_selected_fold,
                          death_ext_xgb_in_hosp_roc_df, death_ext_xgb_in_hosp_resampled_roc_df, death_ext_xgb_in_hosp_resampled_roc_std, xgb_ext_death_in_hosp_roc_auc, death_in_hosp_xgb_selected_fold,
                            ext_death_lstm_in_hosp_roc_df, ext_death_lstm_in_hosp_resampled_roc_df, ext_death_lstm_in_hosp_resampled_roc_std, lstm_ext_death_in_hosp_roc_auc, death_in_hosp_lstm_selected_fold,
                          ext_death_in_hosp_thrivec_resampled_tpr, ext_death_in_hosp_thrivec_resampled_roc_std, ext_death_in_hosp_thrivec_roc_auc,
                          ax, plot_legend = True, tick_label_size = 11, label_font_size = 13)

plt.show()

### Overall Precision-Recall curve

In [ ]:
def plot_pr_curve(transformer_pr_df, transformer_resampled_pr_df, transformer_resampled_pr_std, transformer_pr_aucs, transformer_selected_fold,
                    xgb_pr_df, xgb_resampled_pr_df, xgb_resampled_pr_std, xgb_pr_aucs, xgb_selected_fold,
                    lstm_pr_df, lstm_resampled_pr_df, lstm_resampled_pr_std, lstm_pr_aucs, lstm_selected_fold,
                      thrivec_resampled_precision, thrivec_resampled_pr_std, thrivec_pr_auc,
                      ax1, plot_legend = True, tick_label_size = 11, label_font_size = 13):
    
    all_colors_palette = sns.color_palette(['#f61067', '#049b9a', '#012D98', '#a76dfe'], n_colors=4)
    n_float_numbers = 3


    ## Main model: Transformer
    # plot variation across folds (+/- 1 std)
    error = 1*transformer_resampled_pr_std.values
    lower = transformer_resampled_pr_df[transformer_resampled_pr_df.fold == transformer_selected_fold].precision - error
    upper = transformer_resampled_pr_df[transformer_resampled_pr_df.fold == transformer_selected_fold].precision + error
    ax1.fill_between(transformer_resampled_pr_std.index, lower, upper, alpha=0.2, color=all_colors_palette[0])
    
    # Plot selected fold in bold
    ax1 = sns.lineplot(data=transformer_pr_df[transformer_pr_df.fold == transformer_selected_fold], x='recall', y='precision', color=all_colors_palette[0], label=f'Transformer (area = %0.{n_float_numbers}f)' % transformer_pr_aucs[transformer_selected_fold],
                       ax=ax1, errorbar=None)
    
    ## Comparators: THRIVE-C
    # plot variation across folds (+/- 1 std)
    error = 1*thrivec_resampled_pr_std.values
    lower = thrivec_resampled_precision - error
    upper = thrivec_resampled_precision + error
    ax1.fill_between(np.linspace(0, 1, 200), lower, upper, alpha=0.1, color=all_colors_palette[1])
    
    # Plot THRIVE-C in bold
    sns.lineplot(x=np.linspace(0, 1, 200), y=thrivec_resampled_precision, color=all_colors_palette[1], label='THRIVE-C (area = %0.2f)' % thrivec_pr_auc,
                 ax=ax1, linewidth=2)
    
    ## Comparators: XGBoost
    error = 1*xgb_resampled_pr_std.values
    lower = xgb_resampled_pr_df[xgb_resampled_pr_df.fold == xgb_selected_fold].precision - error
    upper = xgb_resampled_pr_df[xgb_resampled_pr_df.fold == xgb_selected_fold].precision + error
    ax1.fill_between(xgb_resampled_pr_std.index, lower, upper, alpha=0.2, color=all_colors_palette[2])
    ax1 = sns.lineplot(data=xgb_pr_df[xgb_pr_df.fold == xgb_selected_fold], x='recall', y='precision', color=all_colors_palette[2], label=f'XGBoost (area = %0.{n_float_numbers}f)' % xgb_pr_aucs[xgb_selected_fold],
                       ax=ax1, errorbar=None)
    
    ## Comparators: LSTM
    error = 1*lstm_resampled_pr_std.values
    lower = lstm_resampled_pr_df[lstm_resampled_pr_df.fold == lstm_selected_fold].precision - error
    upper = lstm_resampled_pr_df[lstm_resampled_pr_df.fold == lstm_selected_fold].precision + error
    ax1.fill_between(lstm_resampled_pr_std.index, lower, upper, alpha=0.2, color=all_colors_palette[3])
    ax1 = sns.lineplot(data=lstm_pr_df[lstm_pr_df.fold == lstm_selected_fold], x='recall', y='precision', color=all_colors_palette[3], label=f'LSTM (area = %0.{n_float_numbers}f)' % lstm_pr_aucs[lstm_selected_fold],
                       ax=ax1, errorbar=None)
    
    ax1.set_xlabel('Recall', fontsize=label_font_size)
    ax1.set_ylabel('Precision', fontsize=label_font_size)
    ax1.tick_params('x', labelsize=tick_label_size)
    ax1.tick_params('y', labelsize=tick_label_size)
    
    if plot_legend:
        legend_markers, legend_labels = ax1.get_legend_handles_labels()
        sd1_patch = mpatches.Patch(color=all_colors_palette[0], alpha=0.3)
        sd2_patch = mpatches.Patch(color=all_colors_palette[1], alpha=0.3)
        sd3_patch = mpatches.Patch(color=all_colors_palette[2], alpha=0.3)
        sd4_patch = mpatches.Patch(color=all_colors_palette[3], alpha=0.3)
        sd_marker = (sd1_patch, sd2_patch, sd3_patch, sd4_patch)
        sd_labels = '± s.d.'
        legend_markers.append(sd_marker)
        legend_labels.append(sd_labels)
        ax1.legend(legend_markers, legend_labels, fontsize=label_font_size,
                  handler_map={tuple: HandlerTuple(ndivide=None)})
    
    else:
        # remove legend
        ax1.get_legend().remove()
    
    fig1 = ax1.get_figure()
    return fig1

## Plot PR curve for external validation death

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False, 'figure.figsize':(10,10)}
sns.set_theme(style="whitegrid", rc=custom_params, context="paper", font_scale = 1)

ax1 = plt.subplot(111)

fig1 = plot_pr_curve(transformer_ext_death_pr_df, transformer_ext_death_resampled_pr_df, transformer_ext_death_resampled_pr_std, transformer_ext_death_pr_aucs, death_transformer_selected_fold,
                    death_ext_xgb_pr_df, death_ext_xgb_resampled_pr_df, death_ext_xgb_resampled_pr_std, death_ext_xgb_pr_aucs, death_xgb_selected_fold,
                    ext_death_lstm_pr_df, ext_death_lstm_resampled_pr_df, ext_death_lstm_resampled_pr_std, ext_death_lstm_pr_aucs, death_lstm_selected_fold,
                      ext_death_thrivec_resampled_precision, ext_death_thrivec_resampled_pr_std, ext_death_thrivec_pr_auc,
                      ax1, plot_legend = True, tick_label_size = 11, label_font_size = 13)

plt.show()

## Plot PR curve for external validation death in hospital

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False, 'figure.figsize':(10,10)}
sns.set_theme(style="whitegrid", rc=custom_params, context="paper", font_scale = 1)

ax1 = plt.subplot(111)

fig1 = plot_pr_curve(transformer_ext_death_in_hosp_pr_df, transformer_ext_death_in_hosp_resampled_pr_df, transformer_ext_death_in_hosp_resampled_pr_std, transformer_ext_death_in_hosp_pr_aucs, death_in_hosp_transformer_selected_fold,
                    death_ext_xgb_in_hosp_pr_df, death_ext_xgb_in_hosp_resampled_pr_df, death_ext_xgb_in_hosp_resampled_pr_std, death_ext_xgb_in_hosp_pr_aucs, death_in_hosp_xgb_selected_fold,
                    ext_death_lstm_in_hosp_pr_df, ext_death_lstm_in_hosp_resampled_pr_df, ext_death_lstm_in_hosp_resampled_pr_std, ext_death_lstm_in_hosp_pr_aucs, death_in_hosp_lstm_selected_fold,
                      ext_death_in_hosp_thrivec_resampled_precision, ext_death_in_hosp_thrivec_resampled_pr_std, ext_death_in_hosp_thrivec_pr_auc,
                      ax1, plot_legend = True, tick_label_size = 11, label_font_size = 13)

plt.show()

### Combined plot with external validation

In [ ]:
sns.set_theme(style="whitegrid", context="paper", font_scale = 1)

cm = 1/2.54  # centimeters in inches
main_fig2 = plt.figure(figsize=(18 * cm, 20 * cm))

tick_label_size = 6
label_font_size = 7
subplot_number_font_size = 9
suptitle_font_size = 10
plot_subplot_titles = True

subfigs = main_fig2.subfigures(2, 1, wspace=0.07, height_ratios=[1, 1])
    
# External validation death (3 months)
subfigs[0].suptitle('I. Prediction of mortality (3 months)', fontsize=suptitle_font_size, horizontalalignment='left', x=0.1, y=1.0)

ax5, ax6 = subfigs[0].subplots(1, 2)
subfigs[0].subplots_adjust(wspace=0.3)

plot_roc_auc_curve(transformer_ext_death_roc_df, transformer_ext_death_resampled_roc_df, transformer_ext_death_resampled_roc_std, transformer_ext_death_roc_auc, death_transformer_selected_fold,
                    death_ext_xgb_roc_df, death_ext_xgb_resampled_roc_df, death_ext_xgb_resampled_roc_std, xgb_ext_death_roc_auc, death_xgb_selected_fold,
                    ext_death_lstm_roc_df, ext_death_lstm_resampled_roc_df, ext_death_lstm_resampled_roc_std, lstm_ext_death_roc_auc, death_lstm_selected_fold,
                    ext_death_thrivec_resampled_tpr, ext_death_thrivec_resampled_roc_std, thrivec_ext_death_roc_auc,
                    ax5, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax5.set_title('A.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)
    
plot_pr_curve(transformer_ext_death_pr_df, transformer_ext_death_resampled_pr_df, transformer_ext_death_resampled_pr_std, transformer_ext_death_pr_aucs, death_transformer_selected_fold,
                    death_ext_xgb_pr_df, death_ext_xgb_resampled_pr_df, death_ext_xgb_resampled_pr_std, death_ext_xgb_pr_aucs, death_xgb_selected_fold,
                    ext_death_lstm_pr_df, ext_death_lstm_resampled_pr_df, ext_death_lstm_resampled_pr_std, ext_death_lstm_pr_aucs, death_lstm_selected_fold,
                      ext_death_thrivec_resampled_precision, ext_death_thrivec_resampled_pr_std, ext_death_thrivec_pr_auc,
                      ax6, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax6.set_title('B.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)


# External validation death in hospital
subfigs[1].suptitle('II. Prediction of mortality (in hospital)', fontsize=suptitle_font_size, horizontalalignment='left', x=0.1, y=1.0)

ax7, ax8 = subfigs[1].subplots(1, 2)
subfigs[1].subplots_adjust(wspace=0.3)

plot_roc_auc_curve(transformer_ext_death_in_hosp_roc_df, transformer_ext_death_in_hosp_resampled_roc_df, transformer_ext_death_in_hosp_resampled_roc_std, transformer_ext_death_in_hosp_roc_auc, death_in_hosp_transformer_selected_fold,
                    death_ext_xgb_in_hosp_roc_df, death_ext_xgb_in_hosp_resampled_roc_df, death_ext_xgb_in_hosp_resampled_roc_std, xgb_ext_death_in_hosp_roc_auc, death_in_hosp_xgb_selected_fold,
                    ext_death_lstm_in_hosp_roc_df, ext_death_lstm_in_hosp_resampled_roc_df, ext_death_lstm_in_hosp_resampled_roc_std, lstm_ext_death_in_hosp_roc_auc, death_in_hosp_lstm_selected_fold,
                    ext_death_in_hosp_thrivec_resampled_tpr, ext_death_in_hosp_thrivec_resampled_roc_std, ext_death_in_hosp_thrivec_roc_auc,
                    ax7, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax7.set_title('C.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)
    
plot_pr_curve(transformer_ext_death_in_hosp_pr_df, transformer_ext_death_in_hosp_resampled_pr_df, transformer_ext_death_in_hosp_resampled_pr_std, transformer_ext_death_in_hosp_pr_aucs, death_in_hosp_transformer_selected_fold,
                    death_ext_xgb_in_hosp_pr_df, death_ext_xgb_in_hosp_resampled_pr_df, death_ext_xgb_in_hosp_resampled_pr_std, death_ext_xgb_in_hosp_pr_aucs, death_in_hosp_xgb_selected_fold,
                    ext_death_lstm_in_hosp_pr_df, ext_death_lstm_in_hosp_resampled_pr_df, ext_death_lstm_in_hosp_resampled_pr_std, ext_death_lstm_in_hosp_pr_aucs, death_in_hosp_lstm_selected_fold,
                      ext_death_in_hosp_thrivec_resampled_precision, ext_death_in_hosp_thrivec_resampled_pr_std, ext_death_in_hosp_thrivec_pr_auc,
                      ax8, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax8.set_title('D.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)
    
plt.show()

In [ ]:
# main_fig2.savefig(os.path.join(output_dir, 'external_validation_comparative_figure.tiff'), bbox_inches="tight", format='tiff', dpi=1200)


reversed order

In [ ]:
sns.set_theme(style="whitegrid", context="paper", font_scale = 1)

cm = 1/2.54  # centimeters in inches
main_fig2 = plt.figure(figsize=(18 * cm, 20 * cm))

tick_label_size = 6
label_font_size = 7
subplot_number_font_size = 9
suptitle_font_size = 10
plot_subplot_titles = True

subfigs = main_fig2.subfigures(2, 1, wspace=0.07, height_ratios=[1, 1])


# External validation death in hospital
subfigs[0].suptitle('I. Prediction of mortality (in hospital)', fontsize=suptitle_font_size, horizontalalignment='left', x=0.1, y=1.0)

ax7, ax8 = subfigs[0].subplots(1, 2)
subfigs[0].subplots_adjust(wspace=0.3)

plot_roc_auc_curve(transformer_ext_death_in_hosp_roc_df, transformer_ext_death_in_hosp_resampled_roc_df, transformer_ext_death_in_hosp_resampled_roc_std, transformer_ext_death_in_hosp_roc_auc, death_in_hosp_transformer_selected_fold,
                    death_ext_xgb_in_hosp_roc_df, death_ext_xgb_in_hosp_resampled_roc_df, death_ext_xgb_in_hosp_resampled_roc_std, xgb_ext_death_in_hosp_roc_auc, death_in_hosp_xgb_selected_fold,
                    ext_death_lstm_in_hosp_roc_df, ext_death_lstm_in_hosp_resampled_roc_df, ext_death_lstm_in_hosp_resampled_roc_std, lstm_ext_death_in_hosp_roc_auc, death_in_hosp_lstm_selected_fold,
                    ext_death_in_hosp_thrivec_resampled_tpr, ext_death_in_hosp_thrivec_resampled_roc_std, ext_death_in_hosp_thrivec_roc_auc,
                    ax7, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax7.set_title('A.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)
    
plot_pr_curve(transformer_ext_death_in_hosp_pr_df, transformer_ext_death_in_hosp_resampled_pr_df, transformer_ext_death_in_hosp_resampled_pr_std, transformer_ext_death_in_hosp_pr_aucs, death_in_hosp_transformer_selected_fold,
                    death_ext_xgb_in_hosp_pr_df, death_ext_xgb_in_hosp_resampled_pr_df, death_ext_xgb_in_hosp_resampled_pr_std, death_ext_xgb_in_hosp_pr_aucs, death_in_hosp_xgb_selected_fold,
                    ext_death_lstm_in_hosp_pr_df, ext_death_lstm_in_hosp_resampled_pr_df, ext_death_lstm_in_hosp_resampled_pr_std, ext_death_lstm_in_hosp_pr_aucs, death_in_hosp_lstm_selected_fold,
                      ext_death_in_hosp_thrivec_resampled_precision, ext_death_in_hosp_thrivec_resampled_pr_std, ext_death_in_hosp_thrivec_pr_auc,
                      ax8, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax8.set_title('B.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)


# External validation death (3 months)
subfigs[1].suptitle('II. Prediction of mortality (3 months)', fontsize=suptitle_font_size, horizontalalignment='left', x=0.1, y=1.0)

ax5, ax6 = subfigs[1].subplots(1, 2)
subfigs[1].subplots_adjust(wspace=0.3)

plot_roc_auc_curve(transformer_ext_death_roc_df, transformer_ext_death_resampled_roc_df, transformer_ext_death_resampled_roc_std, transformer_ext_death_roc_auc, death_transformer_selected_fold,
                    death_ext_xgb_roc_df, death_ext_xgb_resampled_roc_df, death_ext_xgb_resampled_roc_std, xgb_ext_death_roc_auc, death_xgb_selected_fold,
                    ext_death_lstm_roc_df, ext_death_lstm_resampled_roc_df, ext_death_lstm_resampled_roc_std, lstm_ext_death_roc_auc, death_lstm_selected_fold,
                    ext_death_thrivec_resampled_tpr, ext_death_thrivec_resampled_roc_std, thrivec_ext_death_roc_auc,
                    ax5, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax5.set_title('C.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)
    
plot_pr_curve(transformer_ext_death_pr_df, transformer_ext_death_resampled_pr_df, transformer_ext_death_resampled_pr_std, transformer_ext_death_pr_aucs, death_transformer_selected_fold,
                    death_ext_xgb_pr_df, death_ext_xgb_resampled_pr_df, death_ext_xgb_resampled_pr_std, death_ext_xgb_pr_aucs, death_xgb_selected_fold,
                    ext_death_lstm_pr_df, ext_death_lstm_resampled_pr_df, ext_death_lstm_resampled_pr_std, ext_death_lstm_pr_aucs, death_lstm_selected_fold,
                      ext_death_thrivec_resampled_precision, ext_death_thrivec_resampled_pr_std, ext_death_thrivec_pr_auc,
                      ax6, plot_legend = True, tick_label_size = tick_label_size, label_font_size = label_font_size)

if plot_subplot_titles:
    ax6.set_title('D.', fontsize=subplot_number_font_size, horizontalalignment='left', x=-0.1)


    
plt.show()

In [ ]:
# main_fig2.savefig(os.path.join(output_dir, 'external_validation_comparative_figure_reversed.tiff'), bbox_inches="tight", format='tiff', dpi=1200)